### USE PYG ENVIRONMENT!!!!

In [1]:
# !pip install torch


In [2]:
# Install required packages
import os, torch

# os.environ['TORCH'] = torch.__version__
# !pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
# !pip install git+https://github.com/pyg-team/pytorch_geometric.git

# !pip install sentence_transformers
# !pip3 install fuzzywuzzy[speedup]
# !pip install captum
# !pip install torch-sparse
# !pip install torch-scatter 


In [3]:
### load graph data 
import pickle

with open('../data/graphs/graph.pkl', 'rb') as f:
    data = pickle.load(f)


/Users/christianrasmussen/Documents/thesis/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
### create edge sampling loader
from torch_geometric.data import DataLoader
import torch_geometric.transforms as T
import numpy as np

train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1)(data)


In [20]:
from torch_geometric.loader import LinkNeighborLoader

train_loader = LinkNeighborLoader(
    train_data,
    # Sample 30 neighbors for each node for 2 iterations
    num_neighbors=[10] * 2,
    # Use a batch size of 128 for sampling training nodes
    batch_size=128,
)

val_loader = LinkNeighborLoader(
    val_data,
    num_neighbors=[10] * 2,
    batch_size=128,
)

test_loader = LinkNeighborLoader(
    test_data,
    num_neighbors=[10] * 2,
    batch_size=128,
)
### create model
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(data.num_features, 1)
        self.conv2 = GCNConv(1, 1)
        self.lin = Linear(data.num_features, 1)

    def forward(self, x, edge_index, edge_weight=None):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return self.lin(x)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### train model
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.mse_loss(out, data.y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_nodes

    return total_loss / len(train_loader)

### train model
for epoch in range(1, 101):
    loss = train()
    print('Epoch: {:03d}, Loss: {:.4f}'.format(epoch, loss))


/var/folders/7n/1v9m4ykn66ddbcxt5nwgg9280000gn/T/ipykernel_27077/366039604.py:14: UserWarning: Using a target size (torch.Size([542])) that is different to the input size (torch.Size([542, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out, data.y.view(-1))
/var/folders/7n/1v9m4ykn66ddbcxt5nwgg9280000gn/T/ipykernel_27077/366039604.py:14: UserWarning: Using a target size (torch.Size([574])) that is different to the input size (torch.Size([574, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out, data.y.view(-1))
/var/folders/7n/1v9m4ykn66ddbcxt5nwgg9280000gn/T/ipykernel_27077/366039604.py:14: UserWarning: Using a target size (torch.Size([553])) that is different to the input size (torch.Size([553, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(ou

Epoch: 001, Loss: 229114726.4400
Epoch: 002, Loss: 228443213.3674
Epoch: 003, Loss: 228408125.0751
Epoch: 004, Loss: 228407354.5387
Epoch: 005, Loss: 228407293.8478
Epoch: 006, Loss: 228407287.6487
Epoch: 007, Loss: 228407278.2913
Epoch: 008, Loss: 228407277.4234
Epoch: 009, Loss: 228407274.5272
Epoch: 010, Loss: 228407272.4245
Epoch: 011, Loss: 228407273.0561
Epoch: 012, Loss: 228407272.9764
Epoch: 013, Loss: 228407273.6674
Epoch: 014, Loss: 228407273.6232
Epoch: 015, Loss: 228407271.9332
Epoch: 016, Loss: 228407272.2381
Epoch: 017, Loss: 228407272.0452
Epoch: 018, Loss: 228407271.9863
Epoch: 019, Loss: 228407272.0934
Epoch: 020, Loss: 228407272.4172
Epoch: 021, Loss: 228407272.7391
Epoch: 022, Loss: 228407273.3721
Epoch: 023, Loss: 228407274.0174
Epoch: 024, Loss: 228407273.5465
Epoch: 025, Loss: 228407274.0953
Epoch: 026, Loss: 228407275.0504
Epoch: 027, Loss: 228407274.7687
Epoch: 028, Loss: 228407274.4774
Epoch: 029, Loss: 228407275.3589
Epoch: 030, Loss: 228407275.2499
Epoch: 031

KeyboardInterrupt: 